In [1]:
import pandas as pd
import requests

In [2]:
selected_ticker = 'XOM'

In [3]:
data = pd.DataFrame(pd.date_range(start='2010-01-01', end='2021-02-28', closed=None), columns=['Date'])
print(data.shape)
data.head(2)

(4077, 1)


,Date
0,2010-01-01
1,2010-01-02


In [5]:
path_stock = '../full_dataset/stock_closing_nyse.csv'
path_sentiment_global = '../full_dataset/news-sentiment-index_global.csv' # keyword: exxon, oil, gas
path_sentiment_finance = '../full_dataset/news-sentiment-index_finance.csv' # keyword: exxon, oil, gas
path_stock_index = '../full_dataset/stock_index.csv'
path_price_commodity = '../full_dataset/price_commodity.csv' # filter: crude, gasoline, settle price
path_interest_rate = '../full_dataset/interest_rate.csv' # filter: monthly, exclude forecast
path_inventory_outlook = '../full_dataset/inventory_outlook.csv' # filter: crude, Us, monthly
path_jodi_demand = '../full_dataset/jodi_demand.csv' # filter: jodi, demand
path_mobility_apple = '../full_dataset/mobility_apple.csv' # filter: US, sub-region
path_mobility_google = '../full_dataset/mobility_google.csv' # filter: US


In [11]:
stock_nyse = pd.read_csv(path_stock, parse_dates=['Date Value'])
stock = stock_nyse[stock_nyse.Ticker == selected_ticker][['Date Value','Value']].rename(columns={'Value':'stock_closing_usd', 'Date Value':'Date'}).sort_values('Date')

sentiment_global = pd.read_csv(path_sentiment_global, parse_dates=['DateTime'])
sentiment_global = sentiment_global[['DateTime','Index']].rename(columns={'DateTime':'Date','Index':'sentiment_global_index'}).sort_values('Date')

sentiment_finance = pd.read_csv(path_sentiment_finance, parse_dates=['DateTime'])
sentiment_finance = sentiment_finance[['DateTime','Index']].rename(columns={'DateTime':'Date','Index':'sentiment_finance_index'}).sort_values('Date')

stock_index = pd.read_csv(path_stock_index, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
stock_index = stock_index.pivot_table(index=['Date'], columns='Description', values='Value').reset_index()

price_commodity = pd.read_csv(path_price_commodity, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
price_commodity = price_commodity.pivot_table(index=['Date'], columns='Commodity And Exchange', values='Value').reset_index()

interest_rate = pd.read_csv(path_interest_rate, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
interest_rate['indicator_region'] = interest_rate['Indicator'] + '_' + interest_rate['Region']
interest_rate = interest_rate.pivot_table(index=['Date','Period Value'], columns='indicator_region', values='Value').reset_index().rename(columns={'Period Value' : 'interest_rate_freq'})

inventory_outlook = pd.read_csv(path_inventory_outlook, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
inventory_outlook['sector_unit'] = inventory_outlook['Sub-Sector Level 3'] + '_' + inventory_outlook['Unit']
inventory_outlook = inventory_outlook.pivot_table(index=['Date'], columns='sector_unit', values='Value').reset_index()

jodi_demand = pd.read_csv(path_jodi_demand, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
jodi_demand['indicator'] = 'JODI_demand_' + jodi_demand['Country']
jodi_demand = jodi_demand.pivot_table(index=['Date','Period Value'], columns='indicator', values='Value').reset_index().rename(columns={'Period Value' : 'jodi_demand_freq'})

mobility_apple = pd.read_csv(path_mobility_apple, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
mobility_apple['indicator'] = mobility_apple['Transportation Type'] + '_' + mobility_apple['Country'] + '_' + mobility_apple['Region']
mobility_apple = mobility_apple.pivot_table(index=['Date'], columns='indicator', values='Value').reset_index()

mobility_google_ori = pd.read_csv(path_mobility_google, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
mobility_google = mobility_google_ori.copy()
mobility_google = mobility_google.groupby(['Date','Indicator'])['Value'].agg('mean').reset_index()
mobility_google = mobility_google.pivot_table(index=['Date'], columns='Indicator', values='Value').reset_index()

# mobility_google['indicator'] = mobility_google['Indicator'] + '_' + mobility_google['Sub Region 1'] + '_' + mobility_google['Sub Region 2']
# mobility_google = mobility_google.pivot_table(index=['Date'], columns='indicator', values='Value').reset_index()

In [7]:
jodi_demand

indicator,Date,jodi_demand_freq,JODI_demand_ALGERIA,JODI_demand_ANGOLA,JODI_demand_ARGENTINA,JODI_demand_AUSTRALIA,JODI_demand_AUSTRIA,JODI_demand_AZERBAIJAN,JODI_demand_BAHRAIN,JODI_demand_BANGLADESH,...,JODI_demand_TUNISIA,JODI_demand_TURKEY,JODI_demand_UKRAINE,JODI_demand_UNITED ARAB EMIRATES,JODI_demand_UNITED KINGDOM,JODI_demand_UNITED STATES,JODI_demand_URUGUAY,"JODI_demand_VENEZUELA, BOLIVARIAN REPUBLIC OF",JODI_demand_VIET NAM,JODI_demand_YEMEN
0,2002-01-31,m,114.0,NaN,296.0,767.41,254.93,53.0,NaN,NaN,...,NaN,650.68,NaN,95.0,1577.14,19453.45,36.0,NaN,NaN,NaN
1,2002-02-28,m,128.0,NaN,325.0,844.31,235.16,65.0,NaN,NaN,...,NaN,626.25,NaN,87.0,1665.33,19443.68,43.0,NaN,NaN,NaN
2,2002-03-31,m,125.0,NaN,320.0,782.58,246.88,58.0,NaN,NaN,...,NaN,633.12,NaN,83.0,1693.08,19676.32,37.0,NaN,NaN,NaN
3,2002-04-30,m,128.0,NaN,323.0,814.94,253.45,55.0,NaN,NaN,...,NaN,540.43,NaN,86.0,1632.74,19552.10,41.0,NaN,NaN,NaN
4,2002-05-31,m,120.0,NaN,344.0,801.55,235.16,63.0,NaN,NaN,...,NaN,631.86,NaN,95.0,1627.18,19728.26,37.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,2020-03-01,d,382.0,107.0,518.0,1047.00,239.00,99.0,26.0,NaN,...,NaN,911.00,184.0,NaN,1468.00,18284.00,34.0,NaN,NaN,NaN
219,2020-04-01,d,280.0,95.0,400.0,746.00,208.00,71.0,41.0,NaN,...,NaN,770.00,217.0,NaN,781.00,14733.70,NaN,NaN,NaN,NaN
220,2020-05-01,d,284.0,62.0,456.0,850.00,224.00,87.0,21.0,NaN,...,NaN,704.00,211.0,NaN,981.00,14733.68,NaN,NaN,NaN,NaN
221,2020-06-01,d,NaN,73.0,NaN,911.00,244.00,82.0,29.0,NaN,...,NaN,723.00,226.0,NaN,1042.00,17727.77,NaN,NaN,NaN,NaN


In [8]:
# merge data
data = pd.merge(data, stock, on = ['Date'], how='left')
data = pd.merge(data, sentiment_global, on = ['Date'], how='left')
data = pd.merge(data, sentiment_finance, on = ['Date'], how='left')
data = pd.merge(data, stock_index, on = ['Date'], how='left')
data = pd.merge(data, price_commodity, on = ['Date'], how='left')
data = pd.merge(data, interest_rate, on = ['Date'], how='left')
data = pd.merge(data, inventory_outlook, on = ['Date'], how='left')
data = pd.merge(data, jodi_demand, on = ['Date'], how='left')
data = pd.merge(data, mobility_apple, on = ['Date'], how='left')
data = pd.merge(data, mobility_google, on = ['Date'], how='left')
data

,Date,stock_closing_usd,sentiment_global_index,sentiment_finance_index,DOW JONES COMPOSITE AVERAGE,DOW JONES INDUSTRIAL AVERAGE,DOW JONES TRANSPORTATION AVERAGE,DOW JONES UTILITY AVERAGE,S&P 500,ICE BRENT CRUDE OIL FUTURES,...,WALKING_UNITED STATES_WASHINGTON,WALKING_UNITED STATES_WEST VIRGINIA,WALKING_UNITED STATES_WISCONSIN,WALKING_UNITED STATES_WYOMING,GROCERY AND PHARMACY PERCENT CHANGE FROM BASELINE,PARKS PERCENT CHANGE FROM BASELINE,RESIDENTIAL PERCENT CHANGE FROM BASELINE,RETAIL AND RECREATION PERECENT CHANGE FROM BASELINE,TRANSIT STATIONS PERCENT CHANGE FROM BASELINE,WORKPLACES PERCENT CHANGE FROM BASELINE
0,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,69.15,NaN,NaN,NaN,1.82,NaN,NaN,NaN,80.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-05,69.42,NaN,NaN,NaN,1.82,NaN,NaN,NaN,80.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,2021-02-24,56.70,NaN,NaN,10547.40,31961.86,13630.55,825.65,3925.43,67.04,...,110.53,120.06,144.24,181.79,-5.017857,1.727273,9.901639,-15.596491,-32.545455,-28.145161
4073,2021-02-25,55.76,-0.3,-0.1,10363.89,31402.01,13372.26,814.39,3829.34,66.88,...,108.97,118.30,153.26,193.69,-7.052632,-4.520000,9.688525,-17.224138,-32.176471,-26.790323
4074,2021-02-26,54.37,NaN,-0.2,10231.46,30932.37,13331.27,795.61,3811.15,66.13,...,133.96,132.28,199.61,216.29,-9.228070,0.357143,10.516667,-20.083333,-32.828571,-26.612903
4075,2021-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,146.26,127.74,235.83,219.49,-9.280702,1.689655,5.924528,-17.033333,-32.343750,-12.629032


In [9]:
data.to_csv('../data/data_2010_2021.csv', index=False)

In [47]:
mobility_google_ori = mobility_google_ori.groupby(['Date','Indicator'])['Value'].agg('mean').reset_index()
mobility_google_ori

WORKPLACES PERCENT CHANGE FROM BASELINE                387
PARKS PERCENT CHANGE FROM BASELINE                     387
RETAIL AND RECREATION PERECENT CHANGE FROM BASELINE    387
RESIDENTIAL PERCENT CHANGE FROM BASELINE               387
GROCERY AND PHARMACY PERCENT CHANGE FROM BASELINE      387
TRANSIT STATIONS PERCENT CHANGE FROM BASELINE          387
Name: Indicator, dtype: int64

In [13]:
mobility_google_ori.Indicator.unique()

array(['TRANSIT STATIONS PERCENT CHANGE FROM BASELINE',
       'RESIDENTIAL PERCENT CHANGE FROM BASELINE',
       'PARKS PERCENT CHANGE FROM BASELINE',
       'GROCERY AND PHARMACY PERCENT CHANGE FROM BASELINE',
       'WORKPLACES PERCENT CHANGE FROM BASELINE',
       'RETAIL AND RECREATION PERECENT CHANGE FROM BASELINE'],
      dtype=object)

In [13]:
# inventory_outlook = pd.read_csv(path_inventory_outlook, parse_dates=['Date Value']).rename(columns={'Date Value':'Date'})
inventory_outlook.columns

Index(['Date', 'CRUDE OIL AND LIQUID FUELS SUPPLY_MILLION BARRELS PER DAY',
       'CRUDE OIL INVENTORY (EXCLUDING SPR)_MILLION BARRELS, END-OF-PERIOD',
       'IMPORTED  CRUDE OIL REAL PRICE_REAL DOLLARS PER BARREL',
       'OPEC TOTAL CRUDE OIL PRODUCTION CAPACITY_MILLION BARRELS PER DAY',
       'U.S. CRUDE OIL PRODUCTION_MILLION BARRELS PER DAY'],
      dtype='object', name='sector_unit')